In [15]:
import linecache
import math
import numpy as np
def get_coord(file, num):
    line = linecache.getline(file, num)
    x = float(line[26:38])
    y = float(line[38:46])
    z = float(line[46:54])
    return x, y, z

def direction_cosines(x1, y1, z1, x2, y2, z2):
    d = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    l = (x2 - x1) / d
    m = (y2 - y1) / d
    n = (z2 - z1) / d
    
    return l, m, n

def rotate_point_180(x, y, z, l, m, n):
    rotation_matrix = np.array([
        [2*l**2-1, 2*l*m, 2*l*n],
        [2*l*m, 2*m**2-1, 2*m*n],
        [2*l*n, 2*m*n, 2*n**2-1]
    ])
    point = np.array([[x], [y], [z]])
    rotated_point = np.dot(rotation_matrix, point)
    rotated_x, rotated_y, rotated_z = rotated_point.flatten()
    return rotated_x, rotated_y, rotated_z


def rotate_PDB(filename, l, m, n, outfile):
    inp = open(filename, 'r')
    out = open(outfile, 'w')
    for line in inp:
        if line.startswith('ATOM'):
            init = line[0:26]
            x = float(line[26:38])
            y = float(line[38:46])
            z = float(line[46:54])
            end = line[54:-1]
            (r_x, r_y, r_z) = rotate_point_180(x, y, z, l, m, n)
            ss = '{}{:>12.3f}{:>8.3f}{:>8.3f}{}{}'.format(init, r_x, r_y, r_z, end, "\n")
            out.writelines(ss)
        else:
            out.writelines(line)

x1, y1, z1 = get_coord('repA_dna30A.pdb', 5417)
x2, y2, z2 = get_coord('repA_dna30A.pdb', 4157)

l, m, n = direction_cosines(x1, y1, z1, x2, y2, z2)
rotate_PDB('repA_dna30A.pdb', l, m, n, 'rotated_repA_30A.pdb')


In [18]:
import linecache
import math
def get_coord(file, num):
    line = linecache.getline(file, num)
    x = float(line[26:38])
    y = float(line[38:46])
    z = float(line[46:54])
    return x, y, z

def distance(x1, y1, z1, x2, y2, z2):
    dx = (x2 - x1)
    dy = (y2 - y1)
    dz = (z2 - z1)
    return dx, dy, dz
    
def translate_PDB(filename, tr_x, tr_y, tr_z, outfile):
    inp = open(filename, 'r')
    out = open(outfile, 'w')
    for line in inp:
        if line.startswith('ATOM'):
            init = line[0:26]
            x = float(line[26:38])
            y = float(line[38:46])
            z = float(line[46:54])
            end = line[54:-1]
            n_x = x + tr_x
            n_y = y + tr_y
            n_z = z + tr_z
            ss = '{}{:>12.3f}{:>8.3f}{:>8.3f}{}{}'.format(init, n_x, n_y, n_z, end, "\n")
            out.writelines(ss)
        else:
            out.writelines(line)

x1, y1, z1 = get_coord('repA_dna30A.pdb', 5444)
print(x1, y1, z1)
#x2, y2, z2 = get_coord('rotated_repA_30A.pdb', 5444)
x2, y2, z2 = get_coord('rotated_repA_30A.pdb', 5444)
print(x2,y2,z2)

dx, dy, dz = distance(x2, y2, z2, x1, y1, z1)
print(dx, dy, dz)
translate_PDB('rotated_repA_30A.pdb', dx+0.1, dy+0.1, dz+0.1, 'translated_30_testA.pdb')

-76.661 95.901 122.866
-72.621 -87.821 -131.086
-4.040000000000006 183.72199999999998 253.952


In [19]:
def separate_DNA_Prot(filename, prot_file, DNA_file):
    prot = open(prot_file, 'w')
    dna = open(DNA_file, 'w')
    num_lines = sum(1 for _ in open(filename))
    print(num_lines)
    for i in range(1, 5426):
        line = linecache.getline(filename, i)
        prot.writelines(line)
    for j in range(5426, num_lines+1):
        line = linecache.getline(filename, j)
        dna.writelines(line)   
separate_DNA_Prot('repA_dna30A.pdb', 'repA1.pdb', 'dna30A.pdb')
separate_DNA_Prot('translated_30_testA.pdb', 'repA2.pdb', 'dna30A_t.pdb')

6001
5982


In [25]:
def renumber_DNA(file, outfl):
    num = sum(1 for _ in open(file))
    res_max = int(linecache.getline(file, num-1)[22:26])
    count = 1
    out = open(outfl, 'w')
    for i in range(num, 0, -1):
        line = linecache.getline(file, i)
        if line.startswith('ATOM'):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:22]
            res = int(line[22:26])
            end = line[26:-1]
            new_res = res_max + 1 -res
            new_at = count
            ss = '{}{:>7d}{}{:>4d}{}{}'.format(start, new_at, mid, new_res, end, '\n')
            out.writelines(ss)
            count = count + 1
        else:
            print(line)
renumber_DNA('dna30A_t.pdb', 'dna30A_t_re.pdb')

In [31]:
def combine_DNA(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    print(num1, num2)
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        if line.startswith('ATOM'):
            out.writelines(line)
        else:
            print(line)
    atnum1 = int(linecache.getline(file1, num1-1)[4:11])
    resnum1 = int(linecache.getline(file1, num1-1)[22:26])
    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith('ATOM'):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:22]
            res = int(line[22:26])
            end = line[26:-1]
            n_at = atnum1+j
            n_res = res + resnum1 - 1
            ss = '{}{:>7d}{}{:>4d}{}{}'.format(start, n_at, mid, n_res, end, '\n')
            out.writelines(ss)
        else:
            print(ss)
                   
combine_DNA('dna30A_t_re.pdb', 'dna30A.pdb', 'combined_dna1.pdb')

557 576


In [28]:
def combine_protein(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        out.writelines(line)
    atnum1 = int(linecache.getline(file1, num1)[4:11])
    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith(('ATOM')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' A':
                n_ch = ' D'
            if chain==' B':
                n_ch = ' E'
            if chain==' C':
                n_ch = ' F'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
        if line.startswith(('TER')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' A':
                n_ch = ' D'
            if chain==' B':
                n_ch = ' E'
            if chain==' C':
                n_ch = ' F'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
combine_protein('repA1.pdb', 'repA2.pdb', 'two_RPA.pdb') 

In [32]:
def combine_protein_DNA(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        out.writelines(line)
    atnum1 = int(linecache.getline(file1, num1)[4:11])

    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith(('ATOM')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' D':
                n_ch = ' G'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
        else:
            out.writelines(line)
combine_protein_DNA('two_RPA.pdb', 'combined_dna1.pdb', 'RPA_DNA1.pdb') 

In [7]:
num1 = sum(1 for _ in open('RPA_DNA.pdb'))
import linecache
import math
import numpy as np
x_l = []
y_l = []
z_l = []
for j in range(1, num1+1):
    line = linecache.getline('RPA_DNA.pdb', j)
    if line.startswith(('ATOM')):
        x = line[26:38]
        y = line[38:46]
        z = line[46:54]
        x_l.append(x)
        y_l.append(y)
        z_l.append(z)
print(max(x_l), min(x_l))        

    -108.998      -19.127


In [8]:
def translate_PDB(filename, tr_x, tr_y, tr_z, outfile):
    inp = open(filename, 'r')
    out = open(outfile, 'w')
    for line in inp:
        if line.startswith('ATOM'):
            init = line[0:26]
            x = float(line[26:38])
            y = float(line[38:46])
            z = float(line[46:54])
            end = line[54:-1]
            n_x = x + tr_x
            n_y = y + tr_y
            n_z = z + tr_z
            ss = '{}{:>12.3f}{:>8.3f}{:>8.3f}{}{}'.format(init, n_x, n_y, n_z, end, "\n")
            out.writelines(ss)
        else:
            out.writelines(line)
            
translate_PDB('RPA_DNA.pdb', 64.0, -96.0, -122.0, 'RPA_DNA_TR.pdb')

In [11]:
num1 = sum(1 for _ in open('RPA_DNA_TR.pdb'))
import linecache
import math
import numpy as np
x_l = []
y_l = []
z_l = []
for j in range(1, num1+1):
    line = linecache.getline('RPA_DNA_TR.pdb', j)
    if line.startswith(('ATOM')):
        x = line[26:38]
        y = line[38:46]
        z = line[46:54]
        x_l.append(x)
        y_l.append(y)
        z_l.append(z)
print(max(z_l), min(z_l))  

 -43.663    0.007
